<a href="https://colab.research.google.com/github/Edfred1/Contextual-Music-Crafter/blob/main/CMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation and Setup

This section covers cloning the repository and installing the necessary dependencies.

In [ ]:
# Clone the Git repository
!git clone https://github.com/Edfred1/Contextual-Music-Crafter.git
%cd Contextual-Music-Crafter

# Install dependencies
!pip install -r requirements.txt

### Configuration

Here, we configure the `config.yaml` file. This file contains settings for the AI API, musical parameters, and instrument definitions. You'll need to replace `"YOUR_GOOGLE_AI_API_KEY"` with your actual Google AI API key.

In [ ]:
from ruamel.yaml import YAML
from pathlib import Path

# Path to config.yaml
config_path = Path('config.yaml')

# Load existing config (preserve comments); create a minimal starter if missing
yaml = YAML(typ='rt')
yaml.preserve_quotes = True
yaml.indent(mapping=2, sequence=4, offset=2)

doc = None
if config_path.exists():
    try:
        with config_path.open('r', encoding='utf-8') as f:
            doc = yaml.load(f)
    except Exception:
        doc = None

if doc is None:
    doc = yaml.load("""
api_key:
  - "YOUR_GOOGLE_AI_API_KEY_1"
model_name: "gemini-2.5-pro"
temperature: 0
inspiration: "A new track"
genre: "House"
bpm: 125
key_scale: "C minor"
automation_settings:
  use_pitch_bend: 0
  use_sustain_pedal: 0
  use_cc_automation: 0
  allowed_cc_numbers: [1, 10, 11, 74]
max_output_tokens: 65536
context_window_size: -1
use_call_and_response: 0
number_of_iterations: 1
time_signature:
  beats_per_bar: 4
  beat_value: 4
instruments:
  - name: "Drums"
    program_num: 10
    role: "drums"
  - name: "Bass"
    program_num: 39
    role: "bass"
""")

# Prompt for optional updates without destroying comments
api_keys_input = input("Enter Google AI API key(s), comma-separated (Enter to keep): ").strip()
if api_keys_input:
    keys = [k.strip() for k in api_keys_input.split(',') if k.strip()]
    # Always store as list to support rotation
    doc['api_key'] = keys

model = input(f"Model name [{doc.get('model_name','gemini-2.5-pro')}]: ").strip()
if model:
    doc['model_name'] = model

# Quick edits (press Enter to keep values)
for k in ['inspiration', 'genre', 'bpm', 'key_scale']:
    cur = doc.get(k, '')
    val = input(f"{k} [{cur}]: ").strip()
    if val:
        if k == 'bpm':
            try:
                val = int(val)
            except Exception:
                pass
        doc[k] = val

# Ensure automation block exists and optionally update
auto = doc.get('automation_settings') or {}

def _to01(x):
    try:
        return 1 if int(x) == 1 else 0
    except Exception:
        return 0

if input("Edit automation settings? (y/N): ").strip().lower() == 'y':
    auto['use_pitch_bend'] = _to01(input(f"use_pitch_bend [{auto.get('use_pitch_bend',0)}]: ") or auto.get('use_pitch_bend',0))
    auto['use_sustain_pedal'] = _to01(input(f"use_sustain_pedal [{auto.get('use_sustain_pedal',0)}]: ") or auto.get('use_sustain_pedal',0))
    auto['use_cc_automation'] = _to01(input(f"use_cc_automation [{auto.get('use_cc_automation',0)}]: ") or auto.get('use_cc_automation',0))
    if auto.get('use_cc_automation',0) == 1:
        cc_str = input(f"allowed_cc_numbers {auto.get('allowed_cc_numbers',[1,10,11,74])}: ").strip()
        if cc_str:
            try:
                auto['allowed_cc_numbers'] = [int(x.strip()) for x in cc_str.split(',') if x.strip()]
            except Exception:
                pass

doc['automation_settings'] = auto

# Save back preserving comments
with config_path.open('w', encoding='utf-8') as f:
    yaml.dump(doc, f)

print("config.yaml updated (comments preserved).")

### Running a Python Program from the Repository

This cell lists the Python programs found in the cloned repository and prompts you to select one to execute. The selected program will then be run based on the configuration.

In [ ]:
import os

# Get a list of all files and directories in the cloned repository
files_and_directories = os.listdir('.')

# Filter for Python files
python_files = [f for f in files_and_directories if f.endswith('.py')]

if not python_files:
    print("No Python files found in the repository.")
else:
    print("Found Python files:")
    for i, filename in enumerate(python_files):
        print(f"{i+1}. {filename}")

    # Prompt the user to select a file
    while True:
        try:
            choice = int(input(f"Enter the number of the file to execute (1-{len(python_files)}): "))
            if 1 <= choice <= len(python_files):
                selected_file = python_files[choice - 1]
                print(f"Executing {selected_file}...")
                # Execute the selected Python file
                !python {selected_file}
                break
            else:
                print("Invalid input. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")